# Exercise 5: Lasso Regression on the Franke function  with resampling (Score 10 points))

This exercise is essentially a repeat of the previous two ones, but now
with Lasso regression. Write either your own code (difficult and optional) or, in this case,
you can also use the functionalities of **Scikit-Learn** (recommended). 
Give a
critical discussion of the three methods and a judgement of which
model fits the data best.  Perform here as well an analysis of the bias-variance trade-off using the **bootstrap** resampling technique and an analysis of the mean squared error using cross-validation. 

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import resample
from sklearn.model_selection import  train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from sklearn.preprocessing import StandardScaler


def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4

def FrankeFunctionWithNoise(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    noise = np.random.normal(0, 0.1, len(x)*len(x))
    noise = noise.reshape(len(x),len(x))
    return term1 + term2 + term3 + term4 + noise


def create_X(x, y, n ):
    if len(x.shape) > 1:
        x = np.ravel(x)
        y = np.ravel(y)

    N = len(x)
    l = int((n+1)*(n+2)/2) # Number of elements in beta
    X = np.ones((N,l))

    for i in range(1,n+1):
        q = int((i)*(i+1)/2)
        for k in range(i+1):
            X[:,q+k] = (x**(i-k))*(y**k)
    return X

def R2(y_data, y_model):
    return 1 - np.sum((y_data - y_model) ** 2) / np.sum((y_data - np.mean(y_data)) ** 2)



In [3]:
################# Parameters #################

n = 40 #number of data
list_degree = [10]
k = 5 #number of splits
n_bootstraps = 20
nlambdas = 100
lambdas = np.logspace(-3, 5, nlambdas)

#############################################


#Set up the data
x = np.linspace(0, 1, n)
y = np.linspace(0, 1, n)
x, y = np.meshgrid(x, y)
z = FrankeFunctionWithNoise(x, y)
z = z.ravel()


x_y = np.empty((len(x)*len(x), 2))
x_y[:, 0] = x.ravel()
x_y[:, 1] = y.ravel()

#scale our data
scaler = StandardScaler()
scaler.fit(x_y)
x_y = scaler.transform(x_y)
x_y_train, x_y_test, z_train, z_test = train_test_split(x_y, z.ravel(), test_size=0.2)



Error = np.empty((len(lambdas), len(list_degree)))
#set up our kfolds 
kfold = KFold(n_splits=k)


#For each polynomial degree
for degree in list_degree:
    
    #set up the score for each of kfolds
    scores_KFold = np.zeros((nlambdas, k))
    estimated_mse_sklearn = np.zeros(nlambdas)


    error = np.zeros(lambdas.shape[0])
    bias = np.zeros(lambdas.shape[0])
    variance = np.zeros(lambdas.shape[0])


    j = 0
    
    #set up our polynomial features
    poly = PolynomialFeatures(degree=degree)
    
    for l in lambdas:
        
        #for each lambda set up our lasso model
        lasso_bootstrap = Lasso(alpha=l, fit_intercept=False, max_iter = 100000)
        lasso_cv = Lasso(alpha=l, fit_intercept=False, max_iter = 100000)
        #set up our prediction
        pred_test = np.empty((z_test.shape[0], n_bootstraps))
        pred_train = np.empty((z_train.shape[0], n_bootstraps))

        for i in range(n_bootstraps):

            #for each boostraps resemple our data and compute ou prediction
            x_y_, z_ = resample(x_y_train, z_train)
            X_train = poly.fit_transform(x_y_)
            lasso_bootstrap.fit(X_train, z_)
            Xtest = poly.fit_transform(x_y_test)
            pred_test[:, i] = lasso_bootstrap.predict(Xtest)


        z_pred_kfold  = np.empty((int(z.shape[0] / k), k))
        h = 0

        for train_index, test_index in kfold.split(x_y):
            #Set up our train and test data
            x_train_kfold, x_test_kfold  = x_y[train_index], x_y[test_index]
            z_train_kfold, z_test_kfold = z[train_index], z[test_index]
            #set up our prediction for train and test data
            X_train = poly.fit_transform(x_train_kfold)
            lasso_cv.fit(X_train, z_train_kfold)
            Xtest = poly.fit_transform(x_test_kfold)
            zpred = lasso_cv.predict(Xtest)
            scores_KFold[j, h] = np.sum((zpred - z_test_kfold) ** 2) / np.size(zpred)
            z_pred_kfold[:, h] = zpred
            h += 1
            
        #Compute our score from Cross-validation
        X = poly.fit_transform(x_y)
        
        estimated_mse_folds = cross_val_score(lasso_cv, X, z, scoring='neg_mean_squared_error',cv=kfold)
        estimated_mse_sklearn[j] = np.mean(-estimated_mse_folds)

        #set up our kfolds and the score for each of them 
        error[j] = np.mean(np.mean((z_test.reshape(z_test.shape[0], 1) - pred_test) ** 2, axis=1, keepdims=True))
        bias[j] = np.mean((z_test.reshape(z_test.shape[0], 1) - np.mean(pred_test, axis=1, keepdims=True)) ** 2)
        variance[j] = np.mean(np.var(pred_test, axis=1, keepdims=True))


        print('Polynomial degree:', degree)
        print('Log10(Lambda):',np.log10(l) )
        print("---------Bootstrap---------")
        print('Error:', error[j])
        print('Bias^2:', bias[j])
        print('Var:', variance[j])
        print('{} >= {} + {} = {}'.format(error[j], bias[j], variance[j], bias[j] + variance[j]))
        print("---------Cross Validation---------")
        print('Error_sklearn:', estimated_mse_sklearn[j])
        print("\n\n")

        j +=1

    fig, axs = plt.subplots(2, 1, constrained_layout=True)

    estimated_mse_KFold = np.mean(scores_KFold, axis=1)

    axs[0].plot(np.log10(lambdas), error, label='Error')
    axs[0].plot(np.log10(lambdas), bias, label='bias')
    axs[0].plot(np.log10(lambdas), variance, label='Variance')
    axs[0].set_title(" Boostraps Bias-variance trade-off for polynomial degree = " + str(degree))
    axs[0].set_xlabel("log10(lambda)")
    axs[0].legend()
    axs[0].grid()

    axs[1].plot(np.log10(lambdas), estimated_mse_KFold, label='estimated_mse_KFold')
    axs[1].plot(np.log10(lambdas), estimated_mse_sklearn,'r--',  label='estimated_mse_sklearn')
    axs[1].set_title(" Cross Validation for polynomial degree = " + str(degree))
    axs[1].set_xlabel("log10(lambda)")
    axs[1].legend()
    axs[1].grid()

plt.show()

Polynomial degree: 10
Log10(Lambda): -3.0
---------Bootstrap---------
Error: 0.011373870291715419
Bias^2: 0.010989051038834439
Var: 0.00038481925288097983
0.011373870291715419 >= 0.010989051038834439 + 0.00038481925288097983 = 0.011373870291715419
---------Cross Validation---------
Error_sklearn: 0.5467848366855552



Polynomial degree: 10
Log10(Lambda): -2.919191919191919
---------Bootstrap---------
Error: 0.011369500019543588
Bias^2: 0.011043873845239797
Var: 0.0003256261743037906
0.011369500019543588 >= 0.011043873845239797 + 0.0003256261743037906 = 0.011369500019543588
---------Cross Validation---------
Error_sklearn: 0.38349089144413356





KeyboardInterrupt: 